<div style="text-align: center;">
    <h1><strong>UNIVERSIDAD POLITÉCNICA DE CARTAGENA</strong></h1>
    <h2><strong>GRADO EN CIENCIA E INGENIERÍA DE DATOS</strong></h2>

<div style="display:flex; justify-content:center; align-items:center; padding:5px;">
        <img src="imgs/upct_logo.png" style="height:300px; width:auto">
    </div>

<h2><strong>Procesamiento del Lenguaje Natural</strong></h2>

<h3><strong>PROYECTO:</strong><br>
    <strong>Aplicación PLN para análisis y procesamiento de texto de carácter geopolítico, económico-bursátil y político.</strong></h3>

<p><strong>ESTUDIANTES</strong></p>
    <ul style="list-style-type:none; padding: 0;">
        <li>Rubén Gil Martínez</li>
        <li>Guillermo López Pérez</li>
    </ul>
</div>


### **1) COMPILACIÓN DEL CORPUS DE COMENTARIOS DE REDDIT A TRAVÉS DE SU API Y USO DE PROCESAMIENTO LÉXICO:**

**LIBRERÍAS**

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import datetime as dt
import time

import praw
import pandas as pd
from datetime import datetime
import time
import re
import os


import nltk



In [2]:
# Configuración de credenciales
reddit = praw.Reddit(
    client_id='v4Ozy7so3ZR5n-i4r7Spqw',
    client_secret='iDAtL0KxYY9ky4GyPWyUM2OAZ4LnBg',
    user_agent='dl-pln-2025-RGM-GLP'
)

In [3]:
# Prueba simple de correcta autenticación
try:
    # Intenta acceder a tu nombre de usuario para verificar autenticación
    print(f"Autenticado como: {reddit.user.me()}")
    
    # Prueba una solicitud simple
    for submission in reddit.subreddit("test").hot(limit=1):
        print(f"Título del post de prueba: {submission.title}")
        
    print("¡Autenticación exitosa!")
    
except Exception as e:
    print(f"Error de autenticación: {type(e).__name__} - {str(e)}")
    print("Por favor, verifica tus credenciales y la configuración de la aplicación.")

Autenticado como: None
Título del post de prueba: Some test commands
¡Autenticación exitosa!


In [23]:
# Lista de subreddits a extraer
subreddits = ['Economics', 'books', 'travel', 'gaming', 'EnglishLearning', 'Design']

# Expresiones regulares para filtrado
url_pattern = re.compile(r'(https?://\S+)')
email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')

# Función para verificar si un comentario es válido
def is_valid_comment(text):
    if len(text) < 30:  # Mínimo de 30 caracteres
        return False
    
    # Eliminar URLs y emails
    text_cleaned = url_pattern.sub(' ', text)
    text_cleaned = email_pattern.sub(' ', text_cleaned)
    
    # Verificar si queda suficiente contenido
    return len(text_cleaned.strip()) >= 30

**EXTRACCIÓN DEL CORPUS DE COMENTARIOS EN ARCHIVOS JSON INDIVIDUALES PARA CADA SUBREDDIT**

- 6 subreddits distintos
- 20 hilos distintos de cada subreddit
- Alrededor de 50 comentarios por cada hilo

Durante la extracción hemos tenido en cuenta el evitar duplicados, tomar comentarios distribuidos en el tiempo (hemos tomado desde comentarios de la última semana hasta comentarios de otros años), tomar comentarios que no sean muy cortos y que además no contengan ni urls ni gmails ya que estos no aportan información relevante. 

Parte de este filtrado de comentarios ha sido posible gracias a la función is_valid_comment() previamente definida.

In [24]:
# Procesamiento de cada subreddit
for subreddit_name in subreddits:
    print(f"Procesando subreddit: {subreddit_name}...")
    
    subreddit_data = {
        "subreddit_name": subreddit_name,
        "threads": []
    }

    subreddit = reddit.subreddit(subreddit_name)
    threads_collected = 0  # Contador de hilos extraídos
    extracted_thread_ids = set()  # Para evitar duplicados

    # Diferentes filtros de tiempo
    time_filters = ['all', 'week', 'month', 'year']

    for time_filter in time_filters:
        if threads_collected >= 20:
            break
        
        print(f"  Extrayendo de {subreddit_name} con filtro: {time_filter}")

        try:
            threads = list(subreddit.top(time_filter=time_filter, limit=8))  # Procesamos 8 hilos para cada temporalidad para mayor rapidez y evitar bloqueos
            
            for thread in threads:
                if threads_collected >= 20 or thread.id in extracted_thread_ids:  # Evitar duplicados
                    continue  

                print(f"    Procesando hilo: {thread.title}")

                # Información del hilo
                thread_info = {
                    "thread_id": thread.id,
                    "title": thread.title,
                    "author": str(thread.author),
                    "created_utc": datetime.utcfromtimestamp(thread.created_utc).isoformat(),
                    "score": thread.score,
                    "url": thread.url,
                    "permalink": thread.permalink,
                    "time_filter": time_filter,
                    "subreddit_name": subreddit_name,
                    "comments": []
                }


                # Extraer comentarios
                try:
                    thread.comments.replace_more(limit=3)  # Evita cargar demasiados comentarios
                    flat_comments = thread.comments.list()
                except Exception as e:
                    print(f"      Error al cargar comentarios: {e}")
                    continue

                valid_comments_count = 0

                for comment in flat_comments:
                    if valid_comments_count >= 50:
                        break

                    if is_valid_comment(comment.body): # Si el comentario pasa el filtro de validez es almacenado en la lista de comentarios de su respectivo hilo

                        comment_info = {
                            'comment_id': comment.id,
                            'author': str(comment.author),
                            'text': comment.body,
                            'score': comment.score,
                            'created_utc': datetime.utcfromtimestamp(comment.created_utc).isoformat(),
                            'subreddit_name': subreddit_name,
                            'thread_id': thread.id,
                            'is_submitter': comment.is_submitter,
                            'permalink': comment.permalink
                        }

                        thread_info["comments"].append(comment_info)
                        valid_comments_count += 1

                if valid_comments_count >= 25:  # Mínimo de 25 comentarios válidos, sino, el hilo es descartado
                    subreddit_data["threads"].append(thread_info)
                    extracted_thread_ids.add(thread.id)
                    threads_collected += 1
                    print(f"      Añadido con {valid_comments_count} comentarios válidos")
                else:
                    print(f"      Descartado: solo {valid_comments_count} comentarios válidos")

                time.sleep(1.5)  # Evitar exceso de peticiones a Reddit, nos permite cumplir con las normas de uso de la API y evitar bloqueos temporales

        except Exception as e:
            print(f"Error en {subreddit_name} con filtro {time_filter}: {e}")
            continue

    # Guardar en JSON
    os.makedirs('datasets', exist_ok=True)
    output_file = os.path.join('datasets', f"{subreddit_name}_data_3.0.json")

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(subreddit_data, f, ensure_ascii=False, indent=2)

    print(f"Completado: {subreddit_name} - {threads_collected} hilos guardados en {output_file}\n")

print("Extracción completa.")


Procesando subreddit: Economics...
  Extrayendo de Economics con filtro: all
    Procesando hilo: Economist Warns That Elon Musk Is About to Cause a "Deep, Deep Recession"
      Añadido con 50 comentarios válidos
    Procesando hilo: 65% of Americans support monthly $2,000 COVID stimulus payments, new poll shows
      Añadido con 50 comentarios válidos
    Procesando hilo: Republicans Less Trusted on Economy Than Democrats For First Time in Years
      Añadido con 50 comentarios válidos
    Procesando hilo: Trump's first 50 days mark one of the worst starts for the S&P 500 under any presidency
      Añadido con 50 comentarios válidos
    Procesando hilo: US tourism to suffer huge '£49 billion drop' under Donald Trump
      Añadido con 50 comentarios válidos
    Procesando hilo: Fed report finds 75% of $800 billion Paycheck Protection Program didn't reach employees
      Añadido con 30 comentarios válidos
    Procesando hilo: Trudeau announces Canada ban on Russian oil imports. “Today, 

**PROCESAMIENTO LÉXICO Y CREACIÓN DE UN DATAFRAME PARA TRATAR CON MAYOR FACILIDAD LOS COMENTARIOS**

In [44]:
from symspellpy import SymSpell, Verbosity
import spacy

# Cargamos el modelo de spaCy para realizar la Lematización
nlp = spacy.load('en_core_web_sm')

# Configuración de SymSpell para realizar la corrección automática de errores ortográficos
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = "datasets/frequency_dictionary_en_82_765.txt"  
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1, separator=" ")


True

In [45]:
def remove_emojis(text):
    # Patrón para detectar emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # símbolos & pictogramas
        u"\U0001F680-\U0001F6FF"  # símbolos de transporte & mapas
        u"\U0001F1E0-\U0001F1FF"  # banderas (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

**FUNCIÓN DE PROCESAMIENTO LÉXICO**

In [46]:
def preprocess_text(text, autocorrect=True):
    # Eliminar emojis
    text = remove_emojis(text)
    
    # Convertimos a minúsculas
    text = text.lower()
    
    # Procesamiento con spaCy
    doc = nlp(text)
    
    # Tokenización y eliminación de signos de puntuación y números
    tokens = [token.text for token in doc if not token.is_punct and not token.like_num]
    
    # Corrección ortográfica con SymSpellPy
    if autocorrect:
        corrected_tokens = []
        for token in tokens:
            suggestions = sym_spell.lookup(token, Verbosity.CLOSEST, max_edit_distance=2)
            # Si hay sugerencias, tomamos la primera
            # Si no, mantenemos el token original
            if suggestions:
                corrected_tokens.append(suggestions[0].term)
            else:
                corrected_tokens.append(token)
        tokens = corrected_tokens
    
    # Eliminación de stopwords
    tokens = [token for token in tokens if not token in nlp.Defaults.stop_words]
    
    # Lematización con spaCy
    doc = nlp(' '.join(tokens))
    tokens = [token.lemma_ for token in doc]
    
    return ' '.join(tokens)

## Procesamiento Léxico con spaCy y SymSpellPy

Usamos **spaCy** para el procesamiento completo del texto. Aprovechamos sus capacidades integradas para:

- Tokenización  
- Detección de números y puntuación  
- Uso de su propia lista de *stopwords*  
- Lematización más precisa  

Además, realizamos la **corrección ortográfica** con **SymSpellPy**.

### Ventajas de usar spaCy

El proceso es más eficiente porque spaCy está optimizado para procesamiento de texto. La lematización con spaCy suele ser más precisa que con NLTK porque:

- Considera el contexto de la palabra  
- Tiene mejor manejo de formas irregulares  
- Está más actualizado con el lenguaje moderno  
- Es más rápido en el procesamiento  




**REALIZACIÓN DEL PROCESAMIENTO Y PREPARACIÓN DE LOS COMENTARIOS PARA SU POSTERIOR USO EN DIVERSAS TAREAS**

In [54]:
# Cargamos los datos extraídos y procesamos los comentarios
all_comments = []
dataset_path = 'datasets'

for filename in os.listdir(dataset_path):
    if filename.endswith('_data_3.0.json'):
        with open(os.path.join(dataset_path, filename), 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            for thread in data['threads']:
                for comment in thread['comments']:
                    comment_data = {
                        'body': comment['text'],
                        'score': comment['score'],
                        'created_utc': comment['created_utc'],
                        'thread_title': thread['title'],
                        'subreddit': data['subreddit_name'],
                        'processed_text': preprocess_text(comment['text'], autocorrect=True),
                        'processed_text_no_correction': preprocess_text(comment['text'], autocorrect=False)
                    }
                    all_comments.append(comment_data)

**CREACIÓN DE UN DATAFRAME CON TODOS LOS COMENTARIOS EXTRAÍDOS**

In [55]:
print(len(all_comments))  # Obtención de alrededor de 6000 comentarios válidos

5866


In [56]:
# Crear DataFrame
df_comments = pd.DataFrame(all_comments)

# Mostrar las primeras filas y información del DataFrame
df_comments.head(10)


,body,score,created_utc,thread_title,subreddit,processed_text,processed_text_no_correction
0,"So if you haven't already, there's a bot you c...",2472,2017-11-22T01:50:57,Join the Battle for Net Neutrality!! We need t...,books,bot text help write email fax free charge sena...,bot text help write email fax free charge sena...
1,Aussie here. Please fight for this. If the US ...,697,2017-11-22T03:19:54,Join the Battle for Net Neutrality!! We need t...,books,aussie fight lose net neutrality overflow,aussie fight lose net neutrality overflow
2,Why do we have to do this every few months? 😞,810,2017-11-22T03:17:01,Join the Battle for Net Neutrality!! We need t...,books,month,month
3,💰 The 💰 intent 💰 is 💰 to 💰 provide 💰 consumers...,871,2017-11-22T02:21:54,Join the Battle for Net Neutrality!! We need t...,books,intent provide consumer high quality service s...,intent provide consumer ...
4,Someone suggested that we flood Trumps twitter...,124,2017-11-22T04:41:47,Join the Battle for Net Neutrality!! We need t...,books,suggest flood trump twitter sweet save net neu...,suggest flood trump twitter tweet save net neu...
5,Fucking same shit every month. How many times ...,72,2017-11-22T03:25:33,Join the Battle for Net Neutrality!! We need t...,books,fucking shit month time go try pass shit,fucking shit month time go try pass shit
6,We've only done 266k calls... how? We have mor...,261,2017-11-22T03:13:33,Join the Battle for Net Neutrality!! We need t...,books,ave 266k call editor upcoming caller literally,266k call reditor upvote caller literally
7,“The intent is to provide customers with a sen...,118,2017-11-22T03:29:12,Join the Battle for Net Neutrality!! We need t...,books,intent provide customer sense accomplishment u...,intent provide customer sense accomplishment u...
8,"[To learn about Net Neutrality, why it's impor...",99,2017-11-22T03:13:49,Join the Battle for Net Neutrality!! We need t...,books,learn net neutrality important indoor want too...,learn net neutrality important and/or want too...
9,"Well y'all, I sat through all the prompts and ...",24,2017-11-22T03:36:15,Join the Battle for Net Neutrality!! We need t...,books,sit prompt leave message senator representativ...,y ' sit prompt leave message senator represent...


In [57]:
df_comments.to_csv('datasets/comments_data.csv', index=False, header=True, encoding='utf-8')

In [64]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-100")  # También hay 50, 200 o 300 dimensiones


In [59]:
# Función para obtener el promedio de embeddings de las palabras del texto
def get_mean_embedding(text):
    vectors = [model[word] for word in text.split() if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.nan  # o np.zeros(50) si prefieres un vector nulo

# Aplicamos la función a la columna del DataFrame
df_comments["embedding_mean"] = df_comments["processed_text"].apply(get_mean_embedding)

# Mostramos los primeros resultados
df_comments[["processed_text", "embedding_mean"]].head()


,processed_text,embedding_mean
0,bot text help write email fax free charge sena...,"[-0.11024348, 0.13865672, 0.24223563, -0.19040..."
1,aussie fight lose net neutrality overflow,"[0.27225667, 0.19225435, 0.43482652, -0.350442..."
2,month,"[0.21605, 0.14269, 0.21949, -0.11729, 0.28479,..."
3,intent provide consumer high quality service s...,"[-0.05243463, 0.061616946, 0.26537868, 0.02395..."
4,suggest flood trump twitter sweet save net neu...,"[-0.05058644, -0.0106403595, 0.18712802, -0.19..."


In [60]:
df_comments.head()

,body,score,created_utc,thread_title,subreddit,processed_text,processed_text_no_correction,embedding_mean
0,"So if you haven't already, there's a bot you c...",2472,2017-11-22T01:50:57,Join the Battle for Net Neutrality!! We need t...,books,bot text help write email fax free charge sena...,bot text help write email fax free charge sena...,"[-0.11024348, 0.13865672, 0.24223563, -0.19040..."
1,Aussie here. Please fight for this. If the US ...,697,2017-11-22T03:19:54,Join the Battle for Net Neutrality!! We need t...,books,aussie fight lose net neutrality overflow,aussie fight lose net neutrality overflow,"[0.27225667, 0.19225435, 0.43482652, -0.350442..."
2,Why do we have to do this every few months? 😞,810,2017-11-22T03:17:01,Join the Battle for Net Neutrality!! We need t...,books,month,month,"[0.21605, 0.14269, 0.21949, -0.11729, 0.28479,..."
3,💰 The 💰 intent 💰 is 💰 to 💰 provide 💰 consumers...,871,2017-11-22T02:21:54,Join the Battle for Net Neutrality!! We need t...,books,intent provide consumer high quality service s...,intent provide consumer ...,"[-0.05243463, 0.061616946, 0.26537868, 0.02395..."
4,Someone suggested that we flood Trumps twitter...,124,2017-11-22T04:41:47,Join the Battle for Net Neutrality!! We need t...,books,suggest flood trump twitter sweet save net neu...,suggest flood trump twitter tweet save net neu...,"[-0.05058644, -0.0106403595, 0.18712802, -0.19..."


In [63]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Convertir las listas de 'embedding_padded' en una matriz de NumPy
X = np.vstack(df_comments["embedding_mean"].dropna().values)

# Etiquetas
y = df_comments.loc[df_comments["embedding_mean"].notna(), "subreddit"] # Asegúrate de que las etiquetas estén alineadas con X

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Crear el clasificador Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
# Entrenar el modelo
rf.fit(X_train, y_train)
# Hacer predicciones
y_pred = rf.predict(X_test)
# Evaluar el modelo
print(classification_report(y_test, y_pred, zero_division=0))  # Añadido zero_division=0 para evitar errores en caso de división por cero


                 precision    recall  f1-score   support

         Design       0.62      0.53      0.57       188
      Economics       0.69      0.83      0.76       193
EnglishLearning       0.76      0.65      0.70       198
          books       0.60      0.54      0.57       193
         gaming       0.58      0.62      0.60       200
         travel       0.63      0.68      0.65       199

       accuracy                           0.64      1171
      macro avg       0.65      0.64      0.64      1171
   weighted avg       0.65      0.64      0.64      1171



In [89]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Definir el espacio de búsqueda de hiperparámetros
param_grid = [
    {
        'kernel': ['linear'],
        'C': [0.1, 1, 10]
    },
    {
        'kernel': ['rbf'],
        'C': [0.1, 1, 10],
        'gamma': ['scale', 0.01, 0.1, 1]
    },
    {
        'kernel': ['poly'],
        'C': [0.1, 1, 10],
        'degree': [2, 3, 4],
        'gamma': ['scale', 0.01, 0.1]
    }
]

# Crear el GridSearchCV
grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    verbose=2,
    n_jobs=-1  # Usa todos los núcleos disponibles
)

# Ajustar al set de entrenamiento
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Evaluar el mejor modelo en el set de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nReporte en test set:")
print(classification_report(y_test, y_pred, zero_division=0))


Fitting 5 folds for each of 42 candidates, totalling 210 fits
Mejores parámetros encontrados: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

Reporte en test set:
                 precision    recall  f1-score   support

         Design       0.61      0.64      0.62       188
      Economics       0.72      0.85      0.78       193
EnglishLearning       0.76      0.71      0.73       198
          books       0.68      0.60      0.63       193
         gaming       0.67      0.63      0.65       200
         travel       0.70      0.72      0.71       199

       accuracy                           0.69      1171
      macro avg       0.69      0.69      0.69      1171
   weighted avg       0.69      0.69      0.69      1171

